# Implementation

Author: Jørgen S. Dokken

## Test problem
To solve a test problem, we need to choose the right hand side $f$ and the coefficient $q(u)$ and the boundary $u_D$. Previously, we have worked with manufactured solutions that can  be reproduced without approximation errors. This is more difficult in non-linear porblems, and the algebra is more tedious. Howeve, we will utilize UFLs differentiation capabilities to obtain a manufactured solution. 

For this problem, we will choose $q(u) = 1 + u^2$ and define a two dimensional manufactured solution that is linear in $x$ and $y$:

In [1]:
import ufl
from mpi4py import MPI

from dolfinx.fem import (DirichletBC, Function, FunctionSpace, NonlinearProblem,
                         assemble_scalar, locate_dofs_topological)
from dolfinx.log import set_log_level, LogLevel
from dolfinx.mesh import create_unit_square, locate_entities_boundary
from dolfinx.nls import NewtonSolver

def q(u):
    return 1 + u**2

mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)
x = ufl.SpatialCoordinate(mesh)
u_ufl = 1 + x[0] + 2*x[1]
f = - ufl.div(q(u_ufl)*ufl.grad(u_ufl))

Note that since `x` is a 2D vector, the first component (index 0) resemble $x$, while the second component (index 1) resemble $y$. The resulting function `f` can be directly used in variational formulations in DOLFINx.

As we now have defined our source term and exact solution, we can create the appropriate function space and boundary conditions.
Note that as we have already defined the exact solution, we only have to convert it to a python function that can be evaluated in the interpolation function. We do this by employing the Python `eval` and `lambda`-functions. 

In [2]:
from petsc4py import PETSc
import numpy
V = FunctionSpace(mesh, ("CG", 1))
u_exact = lambda x: eval(str(u_ufl))
u_D = Function(V)
u_D.interpolate(u_exact)
u_D.x.scatter_forward()
fdim = mesh.topology.dim - 1
boundary_facets = locate_entities_boundary(mesh, fdim, lambda x: numpy.full(x.shape[1], True, dtype=bool))
bc = DirichletBC(u_D, locate_dofs_topological(V, fdim, boundary_facets))

We are now ready to define the variational formulation. Note that as the problem is non-linear, we have replace the `TrialFunction` with a `Function`, which serves as the unknown of our problem.

In [3]:
uh = Function(V)
v = ufl.TestFunction(V)
F = q(uh)*ufl.dot(ufl.grad(uh), ufl.grad(v))*ufl.dx - f*v*ufl.dx

The next step is to define the non-linear problem. As it is non-linear we will use [Newtons method](https://en.wikipedia.org/wiki/Newton%27s_method).
Newton's method requires methods for evaluating the residual `F` (including application of boundary conditions), as well as a method for computing the Jacobian matrix. DOLFINx provides the function `NonlinearProblem` that implements these methods. In addition to the boundary conditions, you can supply the variational form for the Jacobian (computed if not supplied), and form and jit parameters, see the [JIT parameters section](../chapter4/compiler_parameters.ipynb).

In [4]:
problem = NonlinearProblem(F, uh, bcs=[bc])

Next, we use the dolfinx Newton solver. We can set the convergence criterions for the solver by changing the absolute tolerance (`atol`), relative tolerance (`rtol`) or the convergence criterion (`residual` or `incremental`).

In [5]:
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6
solver.report = True

We can modify the linear solver in each Newton iteration by accessing the underlying `PETSc` object.

In [6]:
ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "cg"
opts[f"{option_prefix}pc_type"] = "gamg"
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()

We are now ready to solve the non-linear problem. We assert that the solver has converged and print the number of iterations.

In [7]:
set_log_level(LogLevel.INFO)
n, converged = solver.solve(uh)
uh.x.scatter_forward()
assert(converged)
print(f"Number of interations: {n:d}")

Number of interations: 8


2021-12-18 20:36:37.185 (  14.006s) [main            ]  PETScKrylovSolver.cpp:97    INFO| PETSc Krylov solver starting to solve system.
2021-12-18 20:36:37.194 (  14.015s) [main            ]  PETScKrylovSolver.cpp:97    INFO| PETSc Krylov solver starting to solve system.
2021-12-18 20:36:37.198 (  14.019s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 20.3794 (tol = 1e-10) r (rel) = 0.922544(tol = 1e-06)
2021-12-18 20:36:37.198 (  14.019s) [main            ]  PETScKrylovSolver.cpp:97    INFO| PETSc Krylov solver starting to solve system.
2021-12-18 20:36:37.201 (  14.022s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 6.95285 (tol = 1e-10) r (rel) = 0.314745(tol = 1e-06)
2021-12-18 20:36:37.201 (  14.022s) [main            ]  PETScKrylovSolver.cpp:97    INFO| PETSc Krylov solver starting to solve system.
2021-12-18 20:36:37.204 (  14.025s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 4

We observe that the solver converges after $8$ iterations.
If we think of the problem in terms of finite differences on a uniform mesh, $\mathcal{P}_1$ elements mimic standard second-order finite differences, which compute the derivative of a linear or quadratic funtion exactly. Here $\nabla u$ is a constant vector, which is multiplied by $1+u^2$, which is a second order polynomial in $x$ and $y$, which the finite difference operator would compute exactly. We can therefore, even with $\mathcal{P}_1$ elements, expect the manufactured solution to be reproduced by the numerical method. However, if we had chosen a nonlinearity, such as $1+u^4$, this would not be the case, and we would need to verify convergence rates.

In [8]:
# Compute L2 error and error at nodes
V_ex = FunctionSpace(mesh, ("CG", 2))
u_ex = Function(V_ex)
u_ex.interpolate(u_exact)
error_L2 = numpy.sqrt(assemble_scalar((uh - u_ex)**2 * ufl.dx))
print(f"L2-error: {error_L2:.2e}")

# Compute values at mesh vertices
u_vertex_values = uh.compute_point_values()
u_ex_vertex_values = u_ex.compute_point_values()
error_max = numpy.max(numpy.abs(u_vertex_values - u_ex_vertex_values))
print(f"Error_max: {error_max:.2e}")

L2-error: 1.51e-15
Error_max: 5.55e-15


2021-12-18 20:36:41.827 (  18.648s) [main            ]topologycomputation.cpp:612   INFO| Computing mesh entities of dimension 0
2021-12-18 20:36:41.827 (  18.648s) [main            ]topologycomputation.cpp:612   INFO| Computing mesh entities of dimension 1
